In [2]:
import pandas as pd
import pyodbc
username = 'vigrose'
password = 'Hufflepuff10946'
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)

In [3]:
iqvia_query = \
    """ 
    SELECT DISTINCT 
    B.BUSINESS_NAME,
    B.DBA_NAME,
    B.PHONE,
    B.PHYSICAL_ADDR_1,
    B.PHYSICAL_ADDR_2,
    B.PHYSICAL_CITY,
    B.PHYSICAL_STATE,
    B.PHYSICAL_ZIP,
    P.ME,
    T.AFFIL_TYPE_DESC,
    A.AFFIL_IND,
    A.AFFIL_RANK
    FROM 
    ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P, ODS.ODS_IMS_AFFILIATION_TYPE T
    WHERE  
    B.IMS_ORG_ID = A.IMS_ORG_ID
    AND
    A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
    AND
    A.AFFIL_TYPE_ID = T.AFFIL_TYPE_ID
    AND
    P.CURRENT_BATCH_FLAG='Y'
    AND
    A.CURRENT_BATCH_FLAG='Y'
    AND
    B.CURRENT_BATCH_FLAG='Y'
    AND
    B.PHONE IS NOT NULL
    """

In [4]:
symphony_query = \
    """
    SELECT DISTINCT 
    L.OTHER_ID,
    D.ADDR_FRST_TLPHN_NBR,
    D.ADDR_SCND_TLPHN_NBR
    FROM 
    ODS.PRACTITIONER_DEMOGRAPHIC_LAYOUT D, ODS.PRACTITIONER_ADDL_IDS_LAYOUT L
    WHERE  
    D.DS_PRCTR_ID=L.DS_PRCTR_ID
    AND
    D.ADDR_FRST_TLPHN_NBR IS NOT NULL
    AND
    L.ID_QLFR_TYP_CDE=38
    """

In [5]:
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()
symphony = pd.read_sql(con=ODS, sql=symphony_query)
symphony.head()

,OTHER_ID,ADDR_FRST_TLPHN_NBR,ADDR_SCND_TLPHN_NBR
0,0010200001,(727) 642-5087,None
1,0010200002,(843) 792-3537,None
2,0010200003,(828) 254-4337,None
3,0010200005,(504) 842-9618,(504) 842-7444
4,0010200006,(205) 221-7099,None


In [6]:
iqvia = iqvia[['ME','AFFIL_RANK','AFFIL_IND','PHONE']]

In [7]:
dhc = pd.read_csv('../../Data/DHC/DHC_2020-07.csv')

In [8]:
dhc = dhc[['ME', 'Phone Number']]

In [13]:
ppd = pd.read_csv("../../Data/PPD/ppd_data_20200718.csv")

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
PHONES = pd.merge(symphony, iqvia, left_on='OTHER_ID', right_on='ME', how='outer')

TypeError: Can only merge Series or DataFrame objects, a <class 'str'> was passed

In [19]:
dhc['ME'][0]

4201040791.0

In [23]:
ppd =ppd.fillna('None')
PPD = ppd[ppd.TELEPHONE_NUMBER!='None'][['ME','TELEPHONE_NUMBER']]

In [10]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        num = num[:-1]
        nums.append(num)
    return nums
# PPD['ME']=fix_me(PPD.ME)
dhc['ME']=fix_me(dhc.ME)

In [12]:
PHONES

,OTHER_ID,ADDR_FRST_TLPHN_NBR,ADDR_SCND_TLPHN_NBR,ME,AFFIL_RANK,AFFIL_IND,PHONE,Phone Number
0,0010200001,(727) 642-5087,None,0010200001,6.0,A,2053383301,334.213.6255
1,0010200001,(727) 642-5087,None,0010200001,6.0,A,2519493400,334.213.6255
2,0010200001,(727) 642-5087,None,0010200001,6.0,A,3343650651,334.213.6255
3,0010200001,(727) 642-5087,None,0010200001,6.0,A,7275885200,334.213.6255
4,0010200002,(843) 792-3537,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2169735,NaN,NaN,NaN,na,NaN,NaN,NaN,802.847.2700
2169736,NaN,NaN,NaN,na,NaN,NaN,NaN,703.766.2220
2169737,NaN,NaN,NaN,na,NaN,NaN,NaN,920.969.7900
2169738,NaN,NaN,NaN,na,NaN,NaN,NaN,608.263.5660


In [11]:
PHONES = pd.merge(PHONES, dhc, on= 'ME', how='outer')

In [13]:
humach = pd.read_csv("../../Data/Triangulation/wslive.csv")

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
humach = humach[['PHYSICIAN_ME_NUMBER', 'OFFICE_TELEPHONE', 'phone_status']]

In [30]:
PHONES = pd.merge(PHONES, humach, on='ME', how = 'right')

In [21]:
PHONES = PHONES.rename(columns = {'Phone Number': 'Phone_Number'})

In [29]:
humach['ME']=fix_me(humach.PHYSICIAN_ME_NUMBER)

In [43]:
from dataclasses import dataclass, asdict
@dataclass
class phone_stats:
    me: str = 'None'
    telephone: str = 'None'
    iqvia_match: int = 0
    symphony_match: int = 0
    iqvia_preferred: int = 0
    dhc_match: int = 0
    iqvia_present: int = 0
    symphony_present: int = 0
    dhc_present: int = 0
    def score(self):
        total = self.iqvia_match + self.symphony_match + self.dhc_match
        present = self.iqvia_present + self.symphony_present + self.dhc_present
        if present == 0:
            return('None')
        return(total/present)

In [71]:
x = phone_stats()
x.score()

'None'

In [19]:
PHONES = PHONES.fillna('None')

In [28]:
humach =humach.fillna('None')
humach = humach[humach.OFFICE_TELEPHONE!='None']

In [35]:
phones = PHONES[['OTHER_ID', 'ADDR_FRST_TLPHN_NBR', 'ADDR_SCND_TLPHN_NBR', 'ME',
       'AFFIL_RANK', 'AFFIL_IND', 'PHONE', 'Phone_Number',
       'PHYSICIAN_ME_NUMBER_y', 'OFFICE_TELEPHONE_y', 'phone_status_y']]

In [36]:
phones['OFFICE_TELEPHONE_y']

0        2.05748e+09
1        2.56266e+09
2        2.56266e+09
3        3.34357e+09
4        3.34528e+09
            ...     
93087    2.12706e+09
93088    2.01892e+09
93089    8.58652e+09
93090    5.08828e+09
93091    3.23934e+09
Name: OFFICE_TELEPHONE_y, Length: 93092, dtype: object

In [42]:
phones.columns

Index(['OTHER_ID', 'ADDR_FRST_TLPHN_NBR', 'ADDR_SCND_TLPHN_NBR', 'ME',
       'AFFIL_RANK', 'AFFIL_IND', 'PHONE', 'Phone_Number',
       'PHYSICIAN_ME_NUMBER_y', 'OFFICE_TELEPHONE_y', 'phone_status_y'],
      dtype='object')

In [44]:
stats_list = []
for row in phones.itertuples():
    new_stats = phone_stats()
    phone = str(int(row.OFFICE_TELEPHONE_y))
    new_stats.telephone = phone
    new_stats.ME = row.ME
    if row.ADDR_FRST_TLPHN_NBR != 'None':
        new_stats.symphony_present = 1
        symph = row.ADDR_FRST_TLPHN_NBR.replace(')','').replace('(','').replace('-','').replace(' ','')
        if symph == phone:
            new_stats.symphony_match = 1
        elif row.ADDR_SCND_TLPHN_NBR!= 'None':
            symph = row.ADDR_SCND_TLPHN_NBR.replace(')','').replace('(','').replace('-','').replace(' ','')
            if symph == phone:
                new_stats.symphony_match = 1
    if row.PHONE!= 'None':
        new_stats.iqvia_present = 1
        iqvia = row.PHONE
        if iqvia == phone:
            new_stats.iqvia_match = 1
        if row.AFFIL_IND == 'P':
            new_stats.iqvia_preferred = 1
    if row.Phone_Number!= 'None':
        new_stats.dhc_present = 1
        dhc = row.Phone_Number.replace('.','')
        if dhc == phone:
            new_stats.dhc_match = 1
    stats_list.append(new_stats)

In [45]:
score_list = []
for thing in stats_list:
    score_list.append(
        {'PHONE': thing.telephone,
         'ME': thing.ME,
         'SCORE': thing.score()})

In [41]:
stats_list

[phone_stats(telephone='2057480158', iqvia_match=0, symphony_match=1, iqvia_preferred=0, dhc_match=0, iqvia_present=1, symphony_present=1, dhc_present=1),
 phone_stats(telephone='2562659889', iqvia_match=0, symphony_match=1, iqvia_preferred=1, dhc_match=1, iqvia_present=1, symphony_present=1, dhc_present=1),
 phone_stats(telephone='2562659889', iqvia_match=0, symphony_match=1, iqvia_preferred=1, dhc_match=1, iqvia_present=1, symphony_present=1, dhc_present=1),
 phone_stats(telephone='3343569970', iqvia_match=0, symphony_match=0, iqvia_preferred=1, dhc_match=0, iqvia_present=1, symphony_present=1, dhc_present=1),
 phone_stats(telephone='3345283432', iqvia_match=0, symphony_match=0, iqvia_preferred=0, dhc_match=0, iqvia_present=1, symphony_present=1, dhc_present=1),
 phone_stats(telephone='3345283432', iqvia_match=0, symphony_match=0, iqvia_preferred=0, dhc_match=0, iqvia_present=1, symphony_present=1, dhc_present=1),
 phone_stats(telephone='3342894000', iqvia_match=0, symphony_match=0, 

In [50]:
scoress = pd.DataFrame(score_list)

In [49]:
humach['PHONE']=[str(int(x)) for x in humach.OFFICE_TELEPHONE]

In [ ]:
iqvia_query = \
    """ 
    SELECT DISTINCT 
    B.BUSINESS_NAME,
    B.DBA_NAME,
    B.PHONE,
    B.PHYSICAL_ADDR_1,
    B.PHYSICAL_ADDR_2,
    B.PHYSICAL_CITY,
    B.PHYSICAL_STATE,
    B.PHYSICAL_ZIP,
    P.ME,
    T.AFFIL_TYPE_DESC,
    A.AFFIL_IND,
    A.AFFIL_RANK
    FROM 
    ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P, ODS.ODS_IMS_AFFILIATION_TYPE T
    WHERE  
    B.IMS_ORG_ID = A.IMS_ORG_ID
    AND
    A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
    AND
    A.AFFIL_TYPE_ID = T.AFFIL_TYPE_ID
    AND
    P.CURRENT_BATCH_FLAG='Y'
    AND
    A.CURRENT_BATCH_FLAG='Y'
    AND
    B.CURRENT_BATCH_FLAG='Y'
    AND
    B.PHONE IS NOT NULL
    """

In [60]:
SKA = scoress[scoress.SCORE!='None'].sort_values('SCORE', ascending=False).drop_duplicates(['PHONE','ME'], keep='first')

In [61]:
pd.merge(SKA, humach, on=['PHONE', 'ME']).groupby(['phone_status', 'SCORE'])[['PHONE']].count()

PHONE
phone_status SCORE          
Confirmed    0.000000   2547
             0.333333   1851
             0.500000    349
             0.666667   4660
             1.000000   1450
Inconclusive 0.000000    668
             0.333333    209
             0.500000     42
             0.666667    254
             1.000000     98
Known Bad    0.000000   5876
             0.333333    708
             0.500000    246
             0.666667    687
             1.000000    481
No Contact   0.000000  14652
             0.333333   4393
             0.500000   1041
             0.666667   6999
             1.000000   2874
Updated      0.000000    481
             0.333333    150
             0.500000     22
             0.666667    103
             1.000000     33